In [20]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [21]:
datos = pd.read_csv('lp5.data', sep='\t')
datos.head(30)


normal
NaN    -2.0   -1.0   81.0   0.0    -5.0       0.0
                     79.0   0.0    -4.0       0.0
                                   -4.0       0.0
                     80.0   0.0    -4.0       0.0
       -3.0   -1.0   79.0   1.0    -5.0       1.0
       -2.0   -1.0   81.0   0.0    -5.0       0.0
                     80.0   0.0    -4.0       0.0
                     79.0   0.0    -4.0       0.0
                                   -4.0       0.0
                                   -4.0       0.0
                     80.0   1.0    -4.0       0.0
       -1.0   -1.0   80.0   0.0    -4.0       1.0
       -2.0   -1.0   78.0   0.0    -5.0       0.0
       -3.0   -1.0   80.0   1.0    -4.0       1.0
       -2.0   -1.0   79.0   0.0    -4.0       0.0
normal NaN    NaN    NaN   NaN     NaN        NaN
NaN     6.0   -1.0   79.0  -2.0     4.0      -3.0
        42.0  -3.0   80.0   5.0     53.0      3.0
       -5.0    4.0   74.0  -15.0   -10.0     -1.0
       -4.0   -17.0  93.0   26.0   -6.0      -3.0
        2.0    1.0   74.0  -6.0    -3.0      -4.0
        0.0   -5.0   78.0   3.0    -6.0       0.0
       -3.0   -7.0   83.0   8.0    -5.0      -3.0
       -6.0    2.0   71.0  -12.0   -11.0      1.0
       -3.0    3.0   76.0  -6.0    -9.0      -4.0
       -2.0   -9.0   86.0   16.0   -8.0      -2.0
       -4.0    2.0   77.0  -5.0    -7.0       2.0
        2.0   -3.0   81.0   7.0     1.0       9.0
       -1.0   -5.0   80.0   6.0    -6.0       0.0
       -4.0    5.0   78.0  -14.0   -9.0      -4.0